Reference:
- [Approximating Observed Microbiome Data with microbiomeDASim in R](https://github.com/williazo/microbiomeDASim/blob/master/inst/script/mouse_microbiome_approximation.ipynb)

Note: the version of R 4.0.3 at least!

> This dataset is stored as an MRexperiment object with assay data collected at the OTU level. The raw counts represent over 10,000 OTUs that were sequences from a total of 139 samples. These samples represent repeated measurements taken on 12 gnotobiotic mice. All mice were fed the same low-fat, plant polysaccharide–rich diet for the first 21 days of the study. At this point 6 of the mice were then switched to a high-fat, high-sugar “Western” diet. The subseqeuent changes in the microbial community were then observed over a follow-up of roughly 60 days.
> Many of the OTUs are low frequency obserations that do not match out multivariate normal distributional assumptions, especially given our limited sample size (n=12). Therefore, we restrict our analysis to genus level counts rather than OTU level counts, which reduces the feature size from 10,172 to 61.
> We further apply presence and depth filters, and then log normalize the counts to represent our simulated outcome of interest.

So, the taxa values are log-normalized counts.

```R
# install package
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")
BiocManager::install("metagenomeSeq")

# import mouseData from metagenomeSeq
library(metagenomeSeq)
data("mouseData")

# aggregating the counts to the genus level
genus_mouseData <- metagenomeSeq::aggTax(mouseData, lvl="genus")

# additional count filters
genus_mouseData <- filterData(genus_mouseData, present = 10, depth = 1000)
g_lnorm_mat <- MRcounts(genus_mouseData, norm=TRUE, log=TRUE)

# metadata
pd_mouseData <- pData(genus_mouseData)

# save metadata file
write.csv(pd_mouseData, 'C://Users//RDBanjacJe//OneDrive - NESTLE//Jelena//ELMToolBox-Project//notebooks//mousedata_metadata.csv')
# save features data
write.csv(g_lnorm_mat, 'C://Users//RDBanjacJe//OneDrive - NESTLE//Jelena//ELMToolBox-Project//notebooks//mousedata_features.csv')

```

In [3]:
import sys
sys.path.append("C://Users//RDBanjacJe//Desktop//ELMToolBox") 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from elmtoolbox.preprocessing import dataset_bacteria_abundances

In [9]:
# We then combine the observed outcome with phenotype data on the samples which includes their ID, time since study began
df_metadata = pd.read_csv("../INPUT_FILES/mousedata_metadata.csv", index_col=0)
print(df_metadata.shape)
df_metadata.head()

(137, 5)


,mouseID,date,diet,relativeTime,status
PM1:20080108,PM1,2008-01-08,BK,22,0
PM1:20080114,PM1,2008-01-14,BK,28,0
PM1:20071211,PM1,2007-12-11,BK,0,0
PM1:20080121,PM1,2008-01-21,BK,35,0
PM1:20071217,PM1,2007-12-17,BK,6,0


In [10]:
# Many of the OTUs are low frequency obserations that do not match out multivariate normal distributional assumptions, especially given our limited sample size (n=12). Therefore, we restrict our analysis to genus level counts rather than OTU level counts, which reduces the feature size from 10,172 to 61.
# We further apply presence and depth filters, and then log normalize the counts to represent our simulated outcome of interest.
df_features = pd.read_csv("../INPUT_FILES/mousedata_features.csv", index_col=0)
df_features = df_features.T
mapper = {}
for c in df_features.columns:
    mapper[c] = f"g__{c}"
df_features = df_features.rename(mapper=mapper, axis=1)

print(df_features.shape)
df_features.head()

(137, 35)


,g__Akkermansia,g__Alistipes,g__Anaerofilum,g__Anaerofustis,g__Anaerostipes,g__Anaerotruncus,g__Anaerovorax,g__Bacteroides,g__Bilophila,g__Bryantella,...,g__Mogibacterium,g__nan,g__Parabacteroides,g__Prevotella,g__Roseburia,g__RuminococcaceaeIncertaeSedis,g__Ruminococcus,g__Subdoligranulum,g__Sutterella,g__Turicibacter
PM1:20080108,5.198270,8.861973,0.0,0.000000,6.178487,6.178487,5.198270,13.537929,0.000000,7.971544,...,0.0,14.360622,10.713877,14.109796,0.000000,7.488414,7.750109,0.000000,0.000000,0.000000
PM1:20080114,8.163470,8.685500,0.0,0.000000,5.198270,6.496426,0.000000,14.011826,6.178487,8.916306,...,0.0,14.508325,11.041757,8.163470,0.000000,6.756795,9.408481,0.000000,4.237039,0.000000
PM1:20071211,8.536715,6.230243,0.0,0.000000,6.230243,0.000000,0.000000,12.998976,0.000000,0.000000,...,0.0,14.468348,11.044470,13.395906,6.230243,7.220602,0.000000,6.230243,5.249333,5.249333
PM1:20080121,6.658211,8.647458,0.0,0.000000,6.658211,5.101538,5.101538,14.394351,6.080373,8.968667,...,0.0,14.971828,10.942270,11.451726,0.000000,7.872418,9.382984,0.000000,6.658211,0.000000
PM1:20071217,0.000000,9.300649,0.0,5.249333,5.249333,5.249333,0.000000,13.803455,8.215758,5.249333,...,0.0,14.718745,12.505766,15.998821,0.000000,7.220602,8.023806,0.000000,5.249333,0.000000


In [11]:
df = df_features.merge(df_metadata, left_index=True, right_index=True)
df = df.reset_index()
df = df.rename(mapper={"index":"sampleID"}, axis=1)
print(df.shape)
df.head()

(137, 41)


,sampleID,g__Akkermansia,g__Alistipes,g__Anaerofilum,g__Anaerofustis,g__Anaerostipes,g__Anaerotruncus,g__Anaerovorax,g__Bacteroides,g__Bilophila,...,g__RuminococcaceaeIncertaeSedis,g__Ruminococcus,g__Subdoligranulum,g__Sutterella,g__Turicibacter,mouseID,date,diet,relativeTime,status
0,PM1:20080108,5.198270,8.861973,0.0,0.000000,6.178487,6.178487,5.198270,13.537929,0.000000,...,7.488414,7.750109,0.000000,0.000000,0.000000,PM1,2008-01-08,BK,22,0
1,PM1:20080114,8.163470,8.685500,0.0,0.000000,5.198270,6.496426,0.000000,14.011826,6.178487,...,6.756795,9.408481,0.000000,4.237039,0.000000,PM1,2008-01-14,BK,28,0
2,PM1:20071211,8.536715,6.230243,0.0,0.000000,6.230243,0.000000,0.000000,12.998976,0.000000,...,7.220602,0.000000,6.230243,5.249333,5.249333,PM1,2007-12-11,BK,0,0
3,PM1:20080121,6.658211,8.647458,0.0,0.000000,6.658211,5.101538,5.101538,14.394351,6.080373,...,7.872418,9.382984,0.000000,6.658211,0.000000,PM1,2008-01-21,BK,35,0
4,PM1:20071217,0.000000,9.300649,0.0,5.249333,5.249333,5.249333,0.000000,13.803455,8.215758,...,7.220602,8.023806,0.000000,5.249333,0.000000,PM1,2007-12-17,BK,6,0


In [13]:
#df.to_csv("INPUT_FILES/mousedata.xls", sep="\t", index=False)

---

In [14]:
bacteria_names = list(df.columns[df.columns.str.contains("g__")])
len(bacteria_names)

35